# labdevices examples

Below are several examples of how to use or test the drivers in the labdevices folder.
Just scroll through and play around.

## General Tests
Use the pyvisa package connecting to all kinds of different devices via GPIB, SERIAL, USB, ETHERNET

In [6]:
import pyvisa

In [7]:
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
print(resources)

('ASRL/dev/ttyS4::INSTR', 'ASRL/dev/ttyUSB0::INSTR')


In [ ]:
device = rm.open_resource(
    resources[1],
    timeout=100,
    encoding='ascii',
    parity=pyvisa.constants.Parity.none,
    baud_rate=9600,
    data_bits=8,
    stop_bits=pyvisa.constants.StopBits.one,
    flow_control=pyvisa.constants.VI_ASRL_FLOW_NONE,
    write_termination='\r\n',
    read_termination='\r\n'
)


In [ ]:
device.query('AYT')

In [ ]:
device.query('\x05').split(',')

In [ ]:
device.write('\x03')

In [ ]:
device.close()

## Allied Vision

### Manta

In [1]:
from labdevices import allied_vision

In [2]:
allied_vision.get_available_cameras()

[]

In [2]:
ccd = allied_vision.MantaDummy(device_id="DEV_000F314E6DE1")

In [3]:
ccd.initialize()

Connected to camera : DEV_000F314E6DE1


In [26]:
ccd.close()

In [6]:
ccd.acquisition_mode

'Continuous'

In [24]:
pic = ccd.take_single_img()

## ANDO

### Spectrum Analyzer

In [1]:
from labdevices import ando

In [2]:
device = ando.SpectrumAnalyzerDummy('10.0.0.40')

In [3]:
device.initialize()

Connected to ['ANDO dummy']


In [24]:
device.close()

In [21]:
x = list(device.get_x_data())
y = list(device.get_y_data())

## Applied Motion Products
For the control of e.g. the stepper motors of the rotary feedthroughs in the spectrometer chamber.

In [1]:
from labdevices import applied_motion_products

In [2]:
# Available IP addresses are 10.0.0.102 and 10.0.0.103. See also the IP table on the wiki.
device = applied_motion_products.STF03DDummy('10.0.0.103')

In [3]:
device.initialize()

Connected to rotary feedthrough with IP=10.0.0.103.


In [6]:
device.close()

Closed rotary feedthrough with IP=10.0.0.103.


In [7]:
device.query('MV')

'100H179M'

In [4]:
device.set_calibration(360/90)

In [5]:
device._get_future_movement_value()

4.0

In [6]:
device.acceleration

25.0

In [7]:
device.speed

10.0

In [8]:
device.idle_current

0.6

In [9]:
device.move_relative(40)

In [10]:
device.microstep

8

## Keysight

### Oscilloscope

In [11]:
from labdevices import keysight

In [ ]:
device = keysight.Oscilloscope(resources[1])

In [4]:
device.initialize()

Connected to:
KEYSIGHT TECHNOLOGIES,DSO-X 3034T,MY55280218,04.06.2015051201


In [5]:
trace = device.get_trace(1)
print(trace)

(array([-9.531250e-07, -9.527250e-07, -9.523250e-07, ...,  1.045675e-06,
        1.046075e-06,  1.046475e-06]), array([-0.74773869, -0.57889447, -0.64321608, ...,  0.        ,
        0.        ,  0.        ]))


In [5]:
screen = device.get_screen_shot()

In [6]:
with open('screen.png', 'wb+') as f:
    f.write(screen)

In [7]:
device.get_preamble(channel=1)

Preamble(data_format=0, data_type=0, points=5000, count=1, x_increment=4e-10, x_origin=-9.53125e-07, x_reference=0, y_increment=0.008040201, y_origin=0.0, y_reference=128)

### Counter

In [21]:
counter = keysight.Counter('10.0.0.120')

In [22]:
counter.initialize()

Connected to:
Agilent Technologies,53230A,MY50000373,01.12-1313.3321-1.19-4.10-127-154-35


In [4]:
counter.trigger_mode

'IMM'

In [15]:
counter.gate_time

0.001

In [18]:
counter.measure_frequency()

300000.314776433

In [16]:
counter.start_frequency_measurement()

In [19]:
result = counter.query("FETCH?")
print(float(result))

300000.314776433


In [20]:
counter.close()

## Newport

### SMC100

In [1]:
from labdevices import newport

In [2]:
device = newport.SMC100Dummy('/dev/ttyUSB0')

In [3]:
device.initialize()

1ID?
1IDTRA25CC_PN:B183906_UD:18114
Connected to Newport stage 1: TRA25CC_PN:B183906_UD:18114


In [11]:
device.close()

In [4]:
device.idn

1ID?
1IDTRA25CC_PN:B183906_UD:18114


'TRA25CC_PN:B183906_UD:18114'

In [5]:
device.error_and_controller_status()

1TS
1TS01000A


('0100', 10)

In [6]:
device.get_last_command_error()

1TE
1TE@


'@'

In [7]:
device.position

1PA?
1PA0


'0'

In [8]:
device.speed

1VA?
1VA0.4


'0.4'

In [9]:
device.acceleration

1AC?
1AC1.6


'1.6'

## Pfeiffer Vacuum
Like vacuum gauge controllers

In [1]:
from labdevices import pfeiffer_vacuum

In [2]:
sensor = pfeiffer_vacuum.TPG362Dummy()

In [3]:
sensor.initialize()

In [4]:
sensor._device.query('AYT')

'\x06'

In [5]:
sensor.idn

sending: AYT
recv 

receiving:  TPG362,PTG28290,44998061,010300,010100


{'Type': 'TPG362',
 'Model No.': 'PTG28290',
 'Serial No.': '44998061',
 'Firmware version': '010300',
 'Hardware version': '010100'}

In [6]:
sensor.get_gauge_pressure(2)

sending: PR2
recv 

receiving:  5,+0.0000E+00


(0.0, (5, 'No sensor (output: 5,2.0000E-2 [mbar])'))

In [7]:
sensor.temperature

23

In [ ]:
sensor.close()

In [7]:
sensor.get_pressure_unit()

sending: UNI
recv 

receiving:  4


'hPascal'

## Rohde & Schwarz

### Spectrum Analyzer

In [1]:
from labdevices import rohde_schwarz
device = rohde_schwarz.FPC1000('10.0.0.91')
#device = rohde_schwarz.FPC1000(resources[1])

In [22]:
device.initialize()

Connected to:
Rohde&Schwarz,FPC1000,1328.6660K02/102742,V1.50



In [19]:
device.write("INITiate:IMMediate;*WAI")

In [24]:
result = device.query('TRACe:DATA? TRACE1')

In [20]:
type(result[0])

numpy.ndarray

In [25]:
device.close()

Error closing VISA link: socket.timeout: The instrument seems to have stopped responding.


In [8]:
# Acquire the trace and save to file.
# Get trace sometimes times out. I don't know why... Any help is appreciated.
x, y = device.get_trace()
np.savetxt('test.txt', np.column_stack([x,y])) 

In [6]:
device.get_system_alarm()

[(-410, 'Query interrupted'),
 (-410, 'Query interrupted'),
 (-410, 'Query interrupted')]

In [7]:
device.idn

'Rohde&Schwarz,FPC1000,1328.6660K02/102749,V1.50'

### Oscilloscope

In [12]:
from labdevices import rohde_schwarz
device = rohde_schwarz.OscilloscopeDummy('10.0.0.81')

In [13]:
device.initialize()

Connected to:
Rohde&Schwarz dummy


In [14]:
image = device.get_screenshot()

In [15]:
device.get_trace(channel=1)

(array([-3.000e-08, -2.995e-08, -2.990e-08, ...,  2.985e-08,  2.990e-08,
         2.995e-08]),
 array([-0.0345703, -0.0345703, -0.0443359, ..., -0.0345703, -0.0345703,
        -0.0345703]))

In [16]:
device.close()

# Stanford Research Systems

In [1]:
from labdevices import stanford_research_systems

## DG645

In [2]:
dg = stanford_research_systems.DG645('10.0.0.34', 5025)

In [3]:
dg.initialize()

Connected to:
    Stanford Research Systems,DG645,s/n006855,ver1.18.118


In [4]:
dg.get_delay(2)

(0, 7.44498e-05)

In [14]:
dg.get_output_level(2)

'+0.50'

## Thorlabs
Like e.g. the temperature sensor

In [1]:
from labdevices import thorlabs

In [5]:
sensor = thorlabs.TSP01(resources[1])

In [6]:
sensor.initialize()

Connected to Thorlabs,TSP01,M00416749,1.2.0.


In [14]:
sensor.close()

In [7]:
sensor.write('*CLS')

In [13]:
type(sensor.temperature_probe2())

:SENSe4:TEMPerature:DATA?
[21.43771]


float

### Trying to talk to Pfeiffer gauge controllers via Ethernet

In [ ]:
import pyvisa

In [ ]:
rm = pyvisa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
devi = rm.open_resource(
    'TCPIP::192.168.1.10::7776::SOCKET',
    #timeout=100,
    #encoding='ascii',
    #parity=pyvisa.constants.Parity.none,
    #baud_rate=9600,
    #data_bits=8,
    #stop_bits=pyvisa.constants.StopBits.one,
    #flow_control=pyvisa.constants.VI_ASRL_FLOW_NONE,
    write_termination='\r',
    #read_termination='\r\n' 
)

In [ ]:
devi.close()

In [ ]:
devi.query('MV')